In [15]:
%load_ext autoreload
%autoreload 2
import logging
import argparse
import os
import numpy as np
import torch

from ppnp.pytorch import PPNP, PPRGCN
from ppnp.pytorch.training import train_model
from ppnp.pytorch.earlystopping import stopping_args
from ppnp.pytorch.propagation import PPRExact, PPRPowerIteration, DiffusionIteration, PrePPRIteration
from ppnp.data.io import load_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
logging.basicConfig(
        format='%(asctime)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO + 2)

# PyTorch variant
This is not the official code that should be used to reproduce the results from the paper (see `reproduce_results.ipynb` for this), but an adaptation of that code to PyTorch for better accessibility. This notebook reproduces the accuracy of the TensorFlow implementation, but has a longer computation time and varies in some details due to the change to PyTorch.

# Load dataset

In [3]:
graph_name = 'cora_ml'
graph = load_dataset(graph_name)
graph.standardize(select_lcc=True)

<Undirected, unweighted and connected SparseGraph with 15962 edges (no self-loops). Data: adj_matrix (2810x2810), attr_matrix (2810x2879), labels (2810), node_names (2810), attr_names (2879), class_names (7)>

from pathlib import Path
import pickle
import sys
sys.path.append('../gnn-untitled/')

data_dir = Path('../gnn-untitled/pprie/data')
graph_name = 'cora_ml_2c.SG'
path_to_file = data_dir / graph_name

with open(path_to_file, 'rb') as f:
    new_graph = pickle.load(f)
new_graph    

# Set up data splits

First of all we need to decide whether to use the test or validation set. Be mindful that we can only look at the test set exactly _once_ and then can't change any hyperparameters oder model details, no matter what. Everything else would cause overfitting.

In [4]:
test = True

These are the seeds for the dataset splits used in the paper for test/validation.

In [5]:

test_seeds = [
        2144199730,  794209841, 2985733717, 2282690970, 1901557222,
        2009332812, 2266730407,  635625077, 3538425002,  960893189,
        497096336, 3940842554, 3594628340,  948012117, 3305901371,
        3644534211, 2297033685, 4092258879, 2590091101, 1694925034]
# test_seeds = [2144199730, ]

val_seeds = [
        2413340114, 3258769933, 1789234713, 2222151463, 2813247115,
        1920426428, 4272044734, 2092442742, 841404887, 2188879532,
        646784207, 1633698412, 2256863076,  374355442,  289680769,
        4281139389, 4263036964,  900418539,  119332950, 1628837138]

if test:
    seeds = test_seeds
else:
    seeds = val_seeds

Now we can choose the remaining settings for the training/early stopping/validation(test) split. These are the ones chosen in the paper

In [6]:

if graph_name == 'microsoft_academic':
    nknown = 5000
else:
    nknown = 1500
    
idx_split_args = {'ntrain_per_class': 20, 'nstopping': 500, 'nknown': nknown}

# Set up propagation

Next we need to set up the proper pmropagation scheme. In the paper we've introduced the exact PPR propagation used in PPNP and the PPR power iteration propagation used in APPNP.

We use the hyperparameters from the paper.

In [7]:
%%time
if graph_name == 'microsoft_academic':
    alpha = 0.2
else:
    alpha = 0.1

prop_ppnp = PPRExact(graph.adj_matrix, alpha=alpha)
prop_appnp = PPRPowerIteration(graph.adj_matrix, alpha=alpha, niter=10)
# prop_ppnp_d = DiffusionIteration(graph.adj_matrix, niter=10)
prop_ppnp_pre = PrePPRIteration(graph.adj_matrix, alpha=alpha, niter=10)

CPU times: user 17.5 s, sys: 1.39 s, total: 18.9 s
Wall time: 10 s


# Choose model hyperparameters

Now we choose the hyperparameters. These are the ones used in the paper for all datasets.

Note that we choose the propagation for APPNP.

In [22]:
# model_args = {
#     'hiddenunits': [64],
#     'drop_prob': 0.5,
#     'propagation': prop_appnp}

# model_args = {
#     'hiddenunits': [64],
#     'drop_prob': 0.5,
#     'propagation': prop_ppnp_pre}

model_args = {
    'adj_matrix': graph.adj_matrix,
    'hiddenunits': [64],
    'drop_prob': 0.5,
    'niter': 10,
    'alpha': alpha
}

reg_lambda = 5e-3
learning_rate = 0.01

# Train model

First we set the remaining settings for training.

In [23]:
niter_per_seed = 5
save_result = False
print_interval = 10
device = 'cuda'

We use 20 different seeds for splitting and 5 iterations (different random initializations) per split, so we train 100 times altogether. This will take a while.

In [24]:
%%time
results = []
niter_tot = niter_per_seed * len(seeds)
i_tot = 0
for seed in seeds:
    idx_split_args['seed'] = seed
    for _ in range(niter_per_seed):
        i_tot += 1
        logging_string = f"Iteration {i_tot} of {niter_tot}"
        logging.log(22,
                logging_string + "\n                     "
                + '-' * len(logging_string))
        _, result = train_model(
            graph_name, PPRGCN, graph, model_args, learning_rate, reg_lambda,
            idx_split_args, stopping_args, test, device, None, print_interval)
        results.append({})
        results[-1]['stopping_accuracy'] = result['early_stopping']['accuracy']
        results[-1]['valtest_accuracy'] = result['valtest']['accuracy']
        results[-1]['runtime'] = result['runtime']
        results[-1]['runtime_perepoch'] = result['runtime_perepoch']
        results[-1]['split_seed'] = seed

2020-05-06 18:03:52: Iteration 1 of 100
                     ------------------
2020-05-06 18:03:52: PyTorch seed: 2125903515
2020-05-06 18:04:15: Last epoch: 1066, best epoch: 466 (19.822 sec)
2020-05-06 18:04:15: Test accuracy: 82.6%
2020-05-06 18:04:15: Iteration 2 of 100
                     ------------------
2020-05-06 18:04:15: PyTorch seed: 3521875975
2020-05-06 18:04:37: Last epoch: 987, best epoch: 887 (18.367 sec)
2020-05-06 18:04:37: Test accuracy: 82.0%
2020-05-06 18:04:37: Iteration 3 of 100
                     ------------------
2020-05-06 18:04:37: PyTorch seed: 3651986368
2020-05-06 18:05:02: Last epoch: 1197, best epoch: 1016 (22.409 sec)
2020-05-06 18:05:02: Test accuracy: 82.8%
2020-05-06 18:05:02: Iteration 4 of 100
                     ------------------
2020-05-06 18:05:02: PyTorch seed: 1484878049
2020-05-06 18:05:35: Last epoch: 1543, best epoch: 875 (28.726 sec)
2020-05-06 18:05:35: Test accuracy: 83.3%
2020-05-06 18:05:35: Iteration 5 of 100
                

2020-05-06 18:20:52: Last epoch: 982, best epoch: 514 (18.849 sec)
2020-05-06 18:20:52: Test accuracy: 80.7%
2020-05-06 18:20:52: Iteration 36 of 100
                     -------------------
2020-05-06 18:20:52: PyTorch seed: 2435024854
2020-05-06 18:21:14: Last epoch: 982, best epoch: 882 (18.329 sec)
2020-05-06 18:21:14: Test accuracy: 83.4%
2020-05-06 18:21:14: Iteration 37 of 100
                     -------------------
2020-05-06 18:21:14: PyTorch seed: 3232839733
2020-05-06 18:21:44: Last epoch: 1417, best epoch: 1311 (26.374 sec)
2020-05-06 18:21:44: Test accuracy: 81.6%
2020-05-06 18:21:44: Iteration 38 of 100
                     -------------------
2020-05-06 18:21:44: PyTorch seed: 1704124602
2020-05-06 18:22:11: Last epoch: 1227, best epoch: 1041 (23.669 sec)
2020-05-06 18:22:11: Test accuracy: 83.4%
2020-05-06 18:22:11: Iteration 39 of 100
                     -------------------
2020-05-06 18:22:11: PyTorch seed: 1749561096
2020-05-06 18:22:36: Last epoch: 1279, best epoc

2020-05-06 18:36:57: Iteration 70 of 100
                     -------------------
2020-05-06 18:36:57: PyTorch seed: 2868874282
2020-05-06 18:37:27: Last epoch: 1442, best epoch: 1184 (26.854 sec)
2020-05-06 18:37:27: Test accuracy: 86.1%
2020-05-06 18:37:28: Iteration 71 of 100
                     -------------------
2020-05-06 18:37:28: PyTorch seed: 3975494334
2020-05-06 18:37:59: Last epoch: 1471, best epoch: 1371 (27.677 sec)
2020-05-06 18:37:59: Test accuracy: 85.6%
2020-05-06 18:37:59: Iteration 72 of 100
                     -------------------
2020-05-06 18:37:59: PyTorch seed: 2797971516
2020-05-06 18:38:32: Last epoch: 1627, best epoch: 1429 (30.274 sec)
2020-05-06 18:38:32: Test accuracy: 86.2%
2020-05-06 18:38:32: Iteration 73 of 100
                     -------------------
2020-05-06 18:38:32: PyTorch seed: 2496763028
2020-05-06 18:38:59: Last epoch: 1278, best epoch: 1171 (23.827 sec)
2020-05-06 18:38:59: Test accuracy: 86.0%
2020-05-06 18:38:59: Iteration 74 of 100
   

CPU times: user 47min 52s, sys: 5min 22s, total: 53min 15s
Wall time: 48min 9s


# Evaluation

To evaluate the data we use Pandas and Seaborn (for bootstrapping).

In [25]:
import pandas as pd
import seaborn as sns

In [27]:
result_df = pd.DataFrame(results)
result_df.head(20)

,stopping_accuracy,valtest_accuracy,runtime,runtime_perepoch,split_seed
0,0.808,0.825954,19.822443,0.018578,2144199730
1,0.798,0.819847,18.367178,0.018590,2144199730
2,0.802,0.828244,22.409250,0.018706,2144199730
3,0.812,0.832824,28.726454,0.018605,2144199730
4,0.806,0.820611,26.878095,0.018770,2144199730
5,0.814,0.824427,15.202528,0.018722,794209841
6,0.806,0.825191,19.583170,0.018615,794209841
7,0.802,0.819847,23.951620,0.018610,794209841
8,0.794,0.816794,22.617728,0.018554,794209841
9,0.802,0.806870,23.272258,0.018905,794209841


In [28]:
%%time
from ppnp.preprocessing import normalize_attributes
from ppnp.pytorch.training import get_predictions
from ppnp.pytorch.utils import matrix_to_torch

labels_all = graph.labels
attr_mat_norm_np = normalize_attributes(graph.attr_matrix)
attr_mat_norm = matrix_to_torch(attr_mat_norm_np).to(device)

nfeatures = graph.attr_matrix.shape[1]
nclasses = max(labels_all) + 1



CPU times: user 320 ms, sys: 50.3 ms, total: 370 ms
Wall time: 27.1 ms


In [29]:
model_args = {
    'hiddenunits': [64],
    'drop_prob': 0.5,
    'propagation': prop_ppnp_pre}
model = PPNP(nfeatures, nclasses, **model_args).to(device)
%time res = get_predictions(model, attr_mat_norm, torch.arange(len(labels_all)))

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 20 ms


In [222]:
model_args = {
    'hiddenunits': [64],
    'drop_prob': 0.5,
    'propagation': prop_appnp}
model = PPNP(nfeatures, nclasses, **model_args).to(device)
%time res = get_predictions(model, attr_mat_norm, torch.arange(len(labels_all)))

CPU times: user 35.9 ms, sys: 140 µs, total: 36.1 ms
Wall time: 35.9 ms


The standard deviation doesn't really say much about the uncertainty of our results and the standard error of the mean (SEM) assumes a normal distribution. So the best way to get a valid estimate for our results' uncertainty is via bootstrapping.

In [30]:
def calc_uncertainty(values: np.ndarray, n_boot: int = 1000, ci: int = 95) -> dict:
    stats = {}
    stats['mean'] = values.mean()
    boots_series = sns.algorithms.bootstrap(values, func=np.mean, n_boot=n_boot)
    stats['CI'] = sns.utils.ci(boots_series, ci)
    stats['uncertainty'] = np.max(np.abs(stats['CI'] - stats['mean']))
    return stats

In [31]:
stopping_acc = calc_uncertainty(result_df['stopping_accuracy'])
valtest_acc = calc_uncertainty(result_df['valtest_accuracy'])
runtime = calc_uncertainty(result_df['runtime'])
runtime_perepoch = calc_uncertainty(result_df['runtime_perepoch'])

In [32]:
print("APPNP\n"
      "Early stopping: Accuracy: {:.2f} ± {:.2f}%\n"
      "{}: Accuracy: {:.2f} ± {:.2f}%\n"
      "Runtime: {:.3f} ± {:.3f} sec, per epoch: {:.2f} ± {:.2f}ms"
      .format(
          stopping_acc['mean'] * 100,
          stopping_acc['uncertainty'] * 100,
          'Test' if test else 'Validation',
          valtest_acc['mean'] * 100,
          valtest_acc['uncertainty'] * 100,
          runtime['mean'],
          runtime['uncertainty'],
          runtime_perepoch['mean'] * 1e3,
          runtime_perepoch['uncertainty'] * 1e3,
      ))

APPNP
Early stopping: Accuracy: 80.95 ± 0.46%
Test: Accuracy: 83.38 ± 0.32%
Runtime: 25.545 ± 1.067 sec, per epoch: 18.63 ± 0.07ms


original:
APPNP
Early stopping: Accuracy: 80.40 ± 0.00%
Test: Accuracy: 82.90 ± 0.00%
Runtime: 62.598 ± 0.000 sec, per epoch: 41.29 ± 0.00ms

re-run:
APPNP
Early stopping: Accuracy: 81.23 ± 0.42%
Test: Accuracy: 83.60 ± 0.32%
Runtime: 71.608 ± 2.747 sec, per epoch: 42.81 ± 0.37ms

ppnp_d:
A is symmatric, niter=6
APPNP
Early stopping: Accuracy: 77.94 ± 0.59%
Test: Accuracy: 79.70 ± 0.45%
Runtime: 3.912 ± 0.358 sec, per epoch: 20.55 ± 0.15ms

ppnp_d:
A is symmatric, niter=10
APPNP
Early stopping: Accuracy: 78.37 ± 0.62%
Test: Accuracy: 80.38 ± 0.50%
Runtime: 4.173 ± 0.235 sec, per epoch: 21.68 ± 0.12ms

ppnp_pre:
pre-compute PPR matrix (iterative), niter=10
APPNP
Early stopping: Accuracy: 81.34 ± 0.41%
Test: Accuracy: 83.74 ± 0.30%
Runtime: 32.504 ± 1.218 sec, per epoch: 19.06 ± 0.11ms

ppnp_pre:
pre-compute PPR matrix (iterative), niter=20
APPNP
Early stopping: Accuracy: 81.34 ± 0.39%
Test: Accuracy: 83.74 ± 0.33%
Runtime: 32.048 ± 1.240 sec, per epoch: 18.94 ± 0.10ms

PPRGCN:
APPNP
Early stopping: Accuracy: 80.95 ± 0.46%
Test: Accuracy: 83.38 ± 0.32%
Runtime: 25.545 ± 1.067 sec, per epoch: 18.63 ± 0.07ms
